In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 113 rows, 113 columns and 113 nonzeros
Model has 6441 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-03, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 113 rows and 0 columns
Presolve time: 0.02s
Presolved: 0 rows, 113 columns, 0 nonzeros
Presolved model has 6441 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.33402699e+06  0.00000000e+00  9.83e+03 2.46e+02  9.39e+05     0s
   1   3.12082251e+06 -4.79845720e+06  1.47e+03 1.45e+02  2.09e+05     0s
   2   1.77822842e+06 -2.59581717e+06  1.47e-03 1.14e-13  3.87e+04     0s
   3   1.2105

In [2]:
xi_list = GLS_Apr_weekday_PM.xi_list
P = GLS_Apr_weekday_PM.P
L = GLS_Apr_weekday_PM.L  # dimension of xi

113

In [3]:
size(P, 1), size(P, 2)

(56,113)

In [4]:
P

56×113 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [5]:
xi_list

113-element Array{Any,1}:
  498.633 
  179.937 
  566.243 
  427.812 
  139.86  
  312.27  
  165.148 
  475.63  
  141.697 
  154.678 
  424.973 
  131.105 
  109.112 
    ⋮     
   87.7528
   90.6781
   88.1843
   71.9297
  208.949 
  144.733 
  125.162 
 1348.06  
  216.492 
  326.708 
  185.557 
  318.664 

In [6]:
using JuMP

mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    25199
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    18984

Total number of variables............................:     6384
                     variables with only lower bounds:     6384
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [7]:
getvalue(lam)

56-element Array{Float64,1}:
  498.633
  746.18 
  879.942
  782.475
  710.756
  473.329
  261.251
  596.665
  252.181
 1855.39 
  371.634
  220.882
  434.909
    ⋮    
 1071.62 
  829.457
  916.052
 3091.29 
  166.861
  424.74 
  250.792
  478.844
 1348.06 
  216.492
  512.265
  318.664

In [8]:
getobjectivevalue(mGLSJulia)

1.0000000000000138

In [9]:
GLS_Apr_weekday_PM.saveDemandVec(getvalue(lam))

In [10]:
getvalue(p)

56×113 Array{Float64,2}:
 1.0          6.93218e-15  6.9322e-15   …  5.37984e-16  0.0        
 0.0          0.241144     0.758856        0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          1.54554e-15  1.54557e-15     0.0          0.0        
 8.21893e-16  7.24535e-15  7.24537e-15  …  1.17562e-15  0.0        
 3.56506e-15  7.11048e-15  7.11049e-15     3.7603e-15   9.87572e-16
 0.0          5.05591e-15  5.05593e-15     0.0          0.0        
 3.57246e-15  6.99479e-15  6.9948e-15      3.76092e-15  1.08445e-15
 0.0          0.0          0.0             0.0          0.0        
 2.75267e-15  7.7961e-15   7.79612e-15  …  3.03041e-15  0.0        
 3.52858e-15  6.52614e-15  6.52615e-15     3.69366e-15  1.34937e-15
 1.68459e-15  7.58669e-15  7.58671e-15     2.00961e-15  0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [11]:
route_choice_matrix_Apr_PM = getvalue(p)

56×113 Array{Float64,2}:
 1.0          6.93218e-15  6.9322e-15   …  5.37984e-16  0.0        
 0.0          0.241144     0.758856        0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          1.54554e-15  1.54557e-15     0.0          0.0        
 8.21893e-16  7.24535e-15  7.24537e-15  …  1.17562e-15  0.0        
 3.56506e-15  7.11048e-15  7.11049e-15     3.7603e-15   9.87572e-16
 0.0          5.05591e-15  5.05593e-15     0.0          0.0        
 3.57246e-15  6.99479e-15  6.9948e-15      3.76092e-15  1.08445e-15
 0.0          0.0          0.0             0.0          0.0        
 2.75267e-15  7.7961e-15   7.79612e-15  …  3.03041e-15  0.0        
 3.52858e-15  6.52614e-15  6.52615e-15     3.69366e-15  1.34937e-15
 1.68459e-15  7.58669e-15  7.58671e-15     2.00961e-15  0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [12]:
sum(route_choice_matrix_Apr_PM[1, :])

1.0000000000002371

In [13]:
route_choice_matrix_dict_Apr_PM = Dict()

for row_idx = 1:size(route_choice_matrix_Apr_PM)[1]
    for col_idx = 1:size(route_choice_matrix_Apr_PM)[2]
        route_choice_matrix_dict_Apr_PM[(row_idx, col_idx)] = route_choice_matrix_Apr_PM[row_idx, col_idx]
    end
end

In [14]:
route_choice_matrix_dict_Apr_PM

Dict{Any,Any} with 6328 entries:
  (21,13)  => 2.76541e-15
  (53,24)  => 0.0
  (23,106) => 0.0
  (18,103) => 0.0
  (33,111) => 0.0
  (41,34)  => 0.0
  (32,65)  => 0.0
  (56,55)  => 6.27902e-15
  (54,81)  => 2.65598e-14
  (24,108) => 0.0
  (16,73)  => 2.56361e-15
  (1,89)   => 0.0
  (16,67)  => 8.15458e-15
  (10,85)  => 0.0
  (40,97)  => 6.88572e-14
  (1,22)   => 3.72683e-14
  (24,31)  => 0.0
  (6,41)   => 1.73519e-14
  (37,55)  => 3.80604e-15
  (34,113) => 7.89736e-16
  (19,26)  => 0.0
  (50,27)  => 4.0209e-16
  (36,25)  => 0.0
  (9,108)  => 3.29888e-15
  (33,33)  => 0.0
  ⋮        => ⋮

In [15]:
outfile = open("../temp_files/route_choice_CDC_Apr_PM.json", "w")

JSON.print(outfile, route_choice_matrix_dict_Apr_PM)

close(outfile)